In [1]:
%autosave 0

Autosave disabled


In [2]:
import math
import signac
import pandas as pd
import mypackage.code_api as code_api

In [3]:
code = code_api.NameMapping()
rpa = signac.get_project()
print(rpa.root_directory())

selection = rpa.find_jobs({'proton_number': 50, 'neutron_number': 82,
                           'temperature': 0.})

/home/berceanu/Development/random-phase-approximation/signac


In [4]:
myjob = next(selection)

In [5]:
myjob._id

'5eef084526475fef1d287858275f9b28'

https://en.wikipedia.org/wiki/Elementary_charge

In SI, we have the relation:
$$e^2 = 2 \epsilon_0 \alpha h c = 4 \pi \epsilon_0 \alpha \hbar c$$

The conversion factor for charge from SI to Gaussian units is $q_{\text{SI}} = q_{\text{G}} \sqrt{4 \pi \epsilon_{0}^{\text{SI}}}$
https://en.wikipedia.org/wiki/Gaussian_units

Therefore we get
$$ e_{\text{G}}^2 = \alpha \hbar_{\text{G}} c$$

In [6]:
α = 7.297352570e-03 # fine structure constant

ħ_G = 1.0545716e-27 #erg⋅s
erg_to_MeV = 624150.91 # MeV
ħ_MeV = ħ_G * erg_to_MeV # MeV⋅s

c = 2.9979246e+23 # fm/s

In [7]:
ħ_MeVc = ħ_MeV * c
print(f'ħ_G c = {ħ_MeVc} MeV⋅fm')

ħ_G c = 197.32694185813529 MeV⋅fm


In [8]:
e_sq_G = α * ħ_MeVc # MeV⋅fm
print(f'e^2_G = {e_sq_G} MeV⋅fm')

e^2_G = 1.4399642662987042 MeV⋅fm


In [9]:
u_factor = 10 * 16 * math.pi**3 * α / 9 # mb / (e^2 * fm^2)
print('{:.6f}'.format(u_factor))

4.022466


https://doi.org/10.1016/j.nuclphysa.2009.03.009

From Eq. (11) in this paper, we get a factor of 
$$\frac{16\pi^3e^2_G}{9\hbar c} = \frac{16\pi^3 \alpha}{9}$$
$$f_g = \frac{16\pi^3 \alpha}{9} \times S\left[\frac{\text{fm}^2}{\text{MeV}}\right] = 0.402247 \times S\left[\frac{\text{fm}^2}{\text{MeV}}\right] = 4.022466 \times S\left[\frac{\text{mb}}{\text{MeV}}\right] $$

In [10]:
lorvec_fn = myjob.fn(code.out_file(temp='zero', 
                        skalvec='isovector', lorexc='lorentzian'))
excvec_fn = myjob.fn(code.out_file(temp='zero', 
                        skalvec='isovector', lorexc='excitation'))

In [11]:
# ! head -n 25 {lorvec_fn} 
! head -n 20 {excvec_fn}

#RPA-results:
#Nucleus: SN 132
#Excitation: 1 -
#Total number of pairs: 1111
#Number of ph-/ah-pairs: 590/521
#Maximal excitation-energy for particle-hole-pairs: 200
#Maximal excitation-energy for antiparticle-hole-pairs: 2000
#2 imaginary eigenvalues
#Listing imaginary eigenvalues:
#Imaginary part: 3.717608e-01
#Real part: 4.649614e-13
#Imaginary part: -3.717608e-01
#Real part: 4.649614e-13

#Parameterset: DD-ME2
#natural parity: yes
#Isovector result:

7.751396e+00	1.183565e+00
8.046284e+00	8.740788e-02


In [12]:
def sum_rule(fname, 
             cutoff_en=None, # MeV
             unit_factor=u_factor, # mb / (e^2 * fm^2)
            ):
    df = pd.read_csv(fname, delim_whitespace=True, comment='#', skip_blank_lines=True,
            header=None, names=['U', 'fE1']) # Mev, e^2*fm^2/MeV
    if cutoff_en:
        df = df[(df.U <= cutoff_en)] # MeV
    df['fE1s'] = df['fE1'].apply(lambda x: x * unit_factor) # mb/MeV
    df['UxfE1s'] = df.apply(lambda row: (row['U'] * row['fE1s']),
                                   axis=1)
    total_sum = df['UxfE1s'].sum()
    return df, total_sum

In [13]:
lorvec_df, lorvec_sum = sum_rule(lorvec_fn)
excvec_df, excvec_sum = sum_rule(excvec_fn)
print(f'Lorentzian isovector sum rule: {lorvec_sum:.3f} mb.')
print(f'Excitation isovector sum rule: {excvec_sum:.3f} mb.')
print(f'Experimentally measured value is 2330 ± 590 mb.') 

Lorentzian isovector sum rule: 251352.808 mb.
Excitation isovector sum rule: 2548.412 mb.
Experimentally measured value is 2330 ± 590 mb.


In [14]:
lorvec_df.tail()

,U,fE1,UxfE1s
4996,49.96,0.028445,1.421122
4997,49.97,0.028431,1.420699
4998,49.98,0.028418,1.420322
4999,49.99,0.028405,1.419991
5000,50.00,0.028394,1.419706


In [15]:
excvec_df.tail()

,U,fE1,UxfE1s
346,78.39390,0.000434,0.034018
347,78.80869,0.004225,0.332960
348,79.40030,0.000001,0.000090
349,79.73621,0.003079,0.245496
350,79.94843,0.005614,0.448829
